In [1]:
import db_conn
import pandas as pd
import re
pd.options.display.max_columns = None

/home/grace/.local/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
conn = db_conn.get_connection()

## 1. 추출된 전체 부작용
추출된 전체 부작용 중 식별이 가능한 UNIQUE한 약물만을 추려냈습니다.

In [3]:
def get_total_ads():
    cur = conn.cursor()
#     cur.execute("""
#     SELECT DISTINCT(TRIM(LOWER(d.tt_ad))) AS t_ad
#     FROM
#     (SELECT id, tt_ad
#     FROM 
#         (SELECT * 
#         FROM article_table_sentences_m
#         WHERE table_id IN 
#         (SELECT id FROM article_tables WHERE final_topic IS NOT NULL)) M,
#         UNNEST(STRING_TO_ARRAY(t_ad, ' , ')) AS tt_ad
#     WHERE (tt_ad='') is false) as d;
#     """)
    cur.execute("""
    SELECT id, TRIM(lower(d.tt_ad)) AS tt_ad
        FROM
        (SELECT id, tt_ad
        FROM 
            (SELECT * 
            FROM article_table_sentences_m
            WHERE table_id IN 
            (SELECT id FROM article_tables WHERE final_topic IS NOT NULL)) M,
            UNNEST(STRING_TO_ARRAY(t_ad, ' , ')) AS tt_ad
        WHERE (t_ad='') is false) as d;
        """)
    return cur.fetchall()

def get_ad_dictionary():
    cur = conn.cursor()
    cur.execute("""
    SELECT llt_code, pt_code, TRIM(LOWER(llt_name)) AS t_ad
    FROM meddra_llt_181022 M;
    """)
    return cur.fetchall()

In [4]:
ads = pd.DataFrame(get_total_ads())
ads.head()
# ads = pd.read_csv('./distinct_ads.tsv', sep='\t')

,0,1
0,2668800,adverse events
1,2668800,rash
2,2668803,adverse events
3,2668803,dry skin
4,2668802,adverse events


In [5]:
# ads.to_csv('./ads.csv', sep=',')

In [6]:
ads.columns=['id','name']

In [7]:
ads['refined'] = ''
ads['llt_code'] = None
ads['pt_code'] = None
ads.head()

,id,name,refined,llt_code,pt_code
0,2668800,adverse events,,None,None
1,2668800,rash,,None,None
2,2668803,adverse events,,None,None
3,2668803,dry skin,,None,None
4,2668802,adverse events,,None,None


#### 전처리 전 총 부작용의 수는 323398개 입니다

In [8]:
ads.shape

(323398, 5)

### 2. 전처리

#### A. dictionary를 참고하여, 정확히 일치하는 부작용 명은 전처리 없이 포함합니다.

In [9]:
dictionary = pd.DataFrame(get_ad_dictionary())
dictionary.columns = ['llt_code', 'pt_code', 't_ad']
# dictionary = pd.read_csv('./dictionary_ads.tsv', sep='\t')
# dictionary = list(map(lambda x: str(x[0]).lower().replace('"', ''), dictionary))

In [10]:
dictionary.shape

(60939, 3)

In [11]:
dictionary.head()

,llt_code,pt_code,t_ad
0,10000002,10000002,11-beta-hydroxylase deficiency
1,10000007,10000007,17 ketosteroids urine decreased
2,10000010,10000007,17 ketosteroids urine low
3,10000008,10000009,17 ketosteroids urine high
4,10000009,10000009,17 ketosteroids urine increased


### 부작용용어 dictionary와 정확이 매칭하면 추가 전처리 없이 확정

In [12]:
def apply_dictionary():
    for ind, ad in ads[ads['llt_code'].isnull()].iterrows():
    #     print(ad['name'].strip())
        name = ad['name'].strip()
        exact_mathing = None
        exact_mathing = dictionary.loc[dictionary.t_ad==name]

        if len(exact_mathing)==0:
            if name[0] in ['"', '.', ':', '-', '(', ')']:
                name = name[1:]
            if name[len(name)-1] in ['"', '.', ':', '-', '(', ')']:
                name = name[:len(name)-1]
    #         print(name)

            exact_mathing = dictionary.loc[dictionary.t_ad==name]

        if len(exact_mathing)>0:
#             print(exact_mathing.values)
            ads.loc[ind, 'refined'] = name
            ads.loc[ind, 'llt_code'] = exact_mathing.values[0][0]
            ads.loc[ind, 'pt_code'] = exact_mathing.values[0][1]

In [ ]:
apply_dictionary()

In [ ]:
# ads[ads.refined=='apnea']

In [ ]:
print(ads[ads['refined'] == ''].shape)
print(ads[ads['llt_code'].isnull()].shape)

#### A. 부작용을 포함하고 있는 구절이 추출 된 경우, 부작용 외적인 단어들을 제거합니다

## 선생님께서 need_to_refined_ads_hyj.xlsx파일을 통해 부작용명이 아니라고 지정해주신 목록을 추가하였습니다.

In [ ]:
replace_with_space = ['beverages', '<70%', '%', 'values)', '(', ')', '-1987', '-1993', ':', '±', '↑', 'australia-new',
                     'scale-7', 'c-statistics', '-defined', '-derived', '-like', 'like', 'non-beverage',
                     'scale-0', '-days', 'fluorine-18-labelled', '-specific', 'co-variate', '-guided', '≥2°',
                     '≥1', 'composting', 'daids', 'decentralised', 'dmards', 'forage', 'full-scale', 'glues',
                     'granger', 'greyscale', 'grey-scale', 'harvesting', 'immersed', 'invariably',
                     '-guidance', 'lifelon', 'mwaura', 'naito', 'nesting', 'nomad', 'nonsystemi', '≥60', 'primers',
                     'prohibited', 'sanger', '-1993', '<50', '<02', '>heathly']

replace_ends_with = ['-related']

In [ ]:
rep = dict((re.escape(k), ' ') for k in replace_with_space)
pattern = re.compile("|".join(rep.keys()))

for ind, row in ads[ads['refined'] == ''].iterrows():
    refined = ''
    refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
    if refined.strip() == '':
        ads.drop(ind, inplace=True)
    elif row['name'].strip() != refined.strip():
        ads.loc[ind, 'refined'] = refined.strip()
        print(row['name'])
        print(ads.loc[ind]['refined'])
    
    for end in replace_ends_with:
        if row['name'].endswith(end):
            ads.loc[ind, 'refined'] = row['name'].replace(end, '').strip()

ads.reset_index(drop=True)

#### B. 약-부작용 관계를 나타는 것이 아닌 경우, 지웁니다

In [ ]:
non_drug_ad_rel = ['-free', 'free-', ' free ', 'variable', 'wrong drug administered', 'exercise-induced',
                  'underdose', 'switch', 'massage', 'according', '-defining', 'arizona', 'controller', 'athletics',
                  'atopics', 'average', 'bachelor', 'award', '—gasoline', 'cards', 'comer', 'consisting', 'consumer',
                  'courage', 'covaria', 'coverage', 'dimers', 'distinguish', 'eavalues', 'emilia', 'multivaria',
                  'nsaids', 'preexisting', 'reaches', 'recording', 'restaura', 'spain', 'standard', 'tactics',
                  'tattoos', 'tasmania', 'telcagepant', 'testing', 'tragus', 'post-aids', 'scale–parent', 'body-mass-index',
                  'mass-cavity', 'non-melanoma skin', 'single-blind', 'non-comatose', 'non-wheezers', 'high-pitched',
                  'gadofluorine-m', 'anger-control', 'hiv-1-negative', 'pulmonary-systemic', '1apical', 'pre-infarction',
                  'non-trauma', 'stress-mass-heart', 'self-discharge', 'anti-japanese encephalitis', 'non-hypertension',
                   'non-infarct', 'post-stress', 'injury-donor', 'rate–pressure', 'non-neutropenic infections', 'non-hypertrophic',
                   'anti-tachycardia', 'scale-up', '5-fluorouracil', 'isar-smart', '18f-fluoride', 'post-acute myocardial infarction',
                   'hyper-responsiveness', 'pre-aids', 'mid-cavity', 'post-bleeding', 'scale–cognitive', 'double-blind',
                   'high-pressure', 'post-endocarditis', 'peri-procedural complications', 'modulation-blind', 'pre-shock',
                   'pre-angina', 'diphtheria–pertussis–tetanus', 'australia–new', 'q10values', 'pressure-time', 'non-congenital',
                   'anti-hypertensive', 'sepsis-induced', 'snaith-hamilton', 'mass–body', 'pain-to-balloon', 'pain-to-coronary',
                   'non-prostate cancer', 'cold-induced', 'non-melanoma', 'post-hypoxic', 'pre-injury', 'scale-16', 'non-atopic',
                   'non-hypoxic', 'cancer-norfolk', 'langerhans-cell', 'body-mass', 'double-blinding', 'fluorouracil-based',
                   'scale–child', 'fat-mass', 'nifurtimox-eflornithine', 'no adverse events', 'typical'] #'non'

rep = dict((re.escape(k), ' ') for k in non_drug_ad_rel)
pattern = re.compile("|".join(rep.keys()))

In [ ]:
for ind, row in ads.iterrows():
    refined = ''
    refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
    if row['name'].strip() != refined.strip():
        print(row['name'])
        ads.drop(ind, inplace=True)
        print('===============================')

ads.reset_index(drop=True)

In [ ]:
ads.shape

### 파일 중에 부작용이 아니라고 표기해주신 것을 추가로 제거

In [ ]:
replace_excel = pd.read_excel('./need_to_refined_ads_hyj.xlsx')
replace_excel.columns = ['key', 'replace_with', '-']
replace_excel = replace_excel[['key', 'replace_with']]
replace_excel.drop_duplicates(inplace=True)
replace_excel.head()

In [ ]:
replace_excel.shape

In [ ]:
for ind, cura in replace_excel[(replace_excel.replace_with.str.find('부작용 아님')>=0) | (replace_excel.replace_with.str.find('부작용아님')>=0)].iterrows():
#     print(cura['name'])
    need_to_drop = ads[ads.name.str.lower().str.find(cura['key'].lower())>=0]
    for i_n, d_n in need_to_drop.iterrows():
        ads.drop(i_n, inplace=True)
        
ads.reset_index(drop=True)

#### 약-부작용 관계를 나타는 것이 아닌 레코드를 제거하면 274156 남습니다

In [ ]:
ads.shape

### 복잡 부작용, 스펠링 미스 등을 선생님께서 주신 파일로 정제

In [ ]:
def replace_with_manual_curation(ads):
    for ind, row in replace_excel[(replace_excel.replace_with.str.find('부작용 아님')<0) & (replace_excel.replace_with.str.find('부작용아님')<0)].iterrows():
        print(row)
        origin_str = ''
        for d_ind, d_row in ads[ads.llt_code.isnull()].loc[ads.name.str.strip()==row['key']].iterrows():
            origin_str = d_row['name'].replace(row['key'], row['replace_with'].lower()).strip()
            ads.loc[d_ind, 'refined'] = origin_str
        for d_ind, d_row in ads[ads.llt_code.isnull()].loc[ads.refined.str.strip()==row['key']].iterrows():
            origin_str = d_row['refined'].replace(row['key'], row['replace_with'].lower()).strip()
            ads.loc[d_ind, 'refined'] = origin_str
    return ads

ads = replace_with_manual_curation(ads)

In [ ]:
for ind, grp in ads[ads.llt_code.isnull()].groupby('refined'):
#     print(ind)
    found = dictionary.loc[dictionary.t_ad==ind.strip()]
    if len(found)>0:
#         print(found['cui_id'].values[0])
        ads.ix[grp.index.values, 'llt_code'] = found['llt_code'].values[0]
        ads.ix[grp.index.values, 'pt_code'] = found['pt_code'].values[0]

### 여러 부작용이 하나의 레코드에 담겨있는 경우, 분리하여 입력

In [ ]:
def separate_unique_ads(ads):
    additional_ads = []

    for ind, row in ads[(ads.llt_code.isnull()) & (ads.refined.str.find('+')>=0)].iterrows():
        processed = None
        processed = list(map(lambda x: x.strip(), row['refined'].split('+')))

        if len(processed)>0:
            ads.loc[ind, 'refined'] = processed[0]
            for pro in processed[1:]:

                additional_ads.append({'name':row['name'], 'id':row['id'], 'refined':pro})

    for ind, row in ads[(ads.llt_code.isnull()) & (ads.refined.str.find(',')>=0)].iterrows():
        processed = None
        processed = list(map(lambda x: x.strip(), row['refined'].split(',')))

        if len(processed)>0:
            ads.loc[ind, 'refined'] = processed[0]
            for pro in processed[1:]:

                additional_ads.append({'name':row['name'], 'id':row['id'], 'refined':pro})


    ads = ads.append(pd.DataFrame(additional_ads))
#     ads.drop_duplicates(inplace=True)
    return ads

ads = separate_unique_ads(ads)

In [ ]:
ads.shape

In [ ]:
# ads[ads.refined=='apnea']
# ads[ads.llt_code.isnull()]

### 추가 전처리가 필요한 약물 갯수

In [ ]:
len(ads[(ads.refined=='')])

In [ ]:
def get_only_string(text):
    try:
        float(text)
        return False
    except ValueError:
        return True
    
def remove_comparison(name, spliter):
    arrs = list(filter(lambda x: get_only_string(x.strip().replace('%', '')) and x.strip(), name.split(spliter)))
    if len(arrs)>0:
        return arrs
    else:
        return None

ads = ads[~ads.index.duplicated(keep='first')]
for ind, row in ads[ads.refined==''].loc[ads.name.str.find('<')>=0].iterrows():
    processed = remove_comparison(row['name'], '<')
    if processed is None:
        ads.drop(ind, inplace=True)
ads.reset_index(drop=True)

for ind, row in ads[ads.refined==''].loc[ads.name.str.find('<')>=0].iterrows():
    processed = remove_comparison(row['name'], '<')
    if processed is not None:
        ads.loc[ind, 'refined'] = ''.join(processed)

In [ ]:
ads.loc[ads.name.str.find('<')>=0]

여러 부작용이 하나의 레코드에 담겨있는 경우, 분리하여 입력

In [ ]:
additional_ads = []

for ind, row in ads[ads.refined==''].loc[ads.name.str.find('>')>=0].iterrows():
    processed = remove_comparison(row['name'], '>')
    if processed is None:
        ads.drop(ind, inplace=True)
ads.reset_index(drop=True)

for ind, row in ads[ads.refined==''].loc[ads.name.str.find('>')>=0].iterrows():
    processed = remove_comparison(row['name'], '>')
    if processed is not None:
        ads.loc[ind, 'refined'] = processed[0]
        for pro in processed[1:]:
            print(pro)
            additional_ads.append({'name':row['name'], 'id':row['id'], 'refined':pro})

ads = ads.append(pd.DataFrame(additional_ads))
# ads.drop_duplicates(inplace=True)

In [ ]:
ads.shape

In [ ]:
# ads[ads.refined=='apnea']
ads[ads.llt_code.isnull()].shape

### 앞 뒤의 -, . 제거

In [ ]:
remove_list = ['.', '-related', '-', ':', '+', '–', ';', '%', 'time', 'invariably', '||']
def preprocess(name):
    if name[0] in remove_list:
        name = name[1:len(name)]
    
    if name[len(name)-1] in remove_list:
        name = name[0:len(name)-1]
    return name
    
ads.name = ads.name.map(lambda x: preprocess(x))

In [ ]:
remove_exact = ["electrocardiogram changes", "discharged", "historyshock", "alternans", "nonmelanoma skin", "nonmelanoma", "values",
               "statistics", 'actuarial', 'dangerous', 'shockable', 'presence of q waves', 'australia', "topical", "systemic",
               "familial", "fluor", "variations", "towards", "slaughter", "prestroke", "variation", "adjusting", "storage", 
                "subscale", "approached", "reached", "discharges", "accidental", "morale", "australian", "teache", "accidents", 
                "overfitting", "persisting", "confusional", "typically", "existing", "scales", "yards", "typical", "australi", 
                "biomass", "onwards", "subscales", "univariatep", "unrestricted", "rewards", "variant", "mutivariate", "vegetarian", 
                "intolerance", "bivariate", "antibiotics", "adjusted", "cystatin", "hazards", "intense", "anginal", "elispot", 
                "blinded", "pregnant", "pressure", "antipsychotic", "deaths", "fluoropyrimidine", "tolerance", "variance", "cavity", 
                "fluorouracil", "transapical", "resting", "varia", "edwards", "fluorodeoxyglucose", "homocysteine", "teriflunomide", 
                "defibrillation", 'sudden death—not', 'burn—hot', 'beverages'
               ]
for i, row in ads.iterrows():
    if row['name'].strip().lower() in remove_exact:
        ads.drop(i, inplace=True)

ads.reset_index(drop=True)

In [ ]:
apply_dictionary()

In [ ]:
# ads.drop_duplicates(subset=['id', 'llt_code', 'name', 'pt_code', 'refined'], inplace=True)
# ads.reset_index(drop=True)
# ads.head()
ads[ads.refined=='apnea']

In [ ]:
ads = ads[~ads.index.duplicated(keep='first')]
ads = replace_with_manual_curation(ads)

In [ ]:
ads = separate_unique_ads(ads)
ads.reset_index(drop=True)

In [ ]:
for ind, grp in ads[ads.llt_code.isnull()].groupby('refined'):
#     print(ind)
    found = dictionary.loc[dictionary.t_ad==ind.strip()]
    if len(found)>0:
#         print(found['cui_id'].values[0])
        ads.ix[grp.index.values, 'llt_code'] = found['llt_code'].values[0]
        ads.ix[grp.index.values, 'pt_code'] = found['pt_code'].values[0]

In [ ]:
for ind, grp in ads[ads.llt_code.isnull()].groupby('name'):
#     print(ind)
    found = dictionary.loc[dictionary.t_ad==ind.strip()]
    if len(found)>0:
#         print(found['cui_id'].values[0])
        ads.ix[grp.index.values, 'refined'] = ind.strip()
        ads.ix[grp.index.values, 'llt_code'] = found['llt_code'].values[0]
        ads.ix[grp.index.values, 'pt_code'] = found['pt_code'].values[0]

In [ ]:
# ads[(ads.llt_code=='') | (ads.pt_code=='')]['refined'].tolist()
# 'illness', 'wind', 'sting','tics','nstemi','herniation','suffering', 'fasting','carcinomtous rash','apoptosis',
irrelevant_words = ['withdrawn', 'non-ischaemia', 'illness', 'wind', 'values', 'sting', 'pressure', 'nstemi', 'rage',\
 'hanging', 'windy', 'tics', 'herniation','suffering', 'fasting','carcinomtous rash','apoptosis', 'term birth']
for ind, row in ads[(ads.llt_code.isnull()) | (ads.pt_code.isnull())].iterrows():
    refined = row['refined']
    if refined.strip() in irrelevant_words:
        ads.drop(ind, inplace=True)

In [ ]:
ads.reset_index(drop=True)

In [ ]:
# ads[ads.llt_code.isnull()].refined = ads[ads.llt_code.isnull()].refined.apply(lambda x: x.strip())

In [ ]:
ads[ads.llt_code.isnull()]

In [ ]:
# ads[ads.refined=='apnea']

In [ ]:
ads[ads.llt_code.isnull()].refined.unique()#.shape

In [ ]:
# ads[(ads.llt_code=='') | (ads.pt_code.str=='')]
ads.isnull().sum()

In [ ]:
ads.shape

In [ ]:
ads[~ads.llt_code.isnull()].shape

## Bulk insert

In [ ]:
import os
from io import StringIO

In [ ]:
ads.head()

In [ ]:
ads.columns

In [ ]:
ads.columns = ['sent_id', 'llt_code', 'name', 'pt_code', 'refined']
ads.head()

In [ ]:
sio = StringIO()
sio.write(ads.to_csv(index=None, header=None))
sio.seek(0)

with conn.cursor() as c:
    c.copy_from(sio, "table_extracted_ad", columns = ads.columns, sep=',')
    conn.commit()

In [ ]:
# ads[(ads.llt_code=='') | (ads.pt_code.str=='')]
#ads[(ads.id.isnull())]
ads.shape

In [ ]:
ads[ads.refined=='apnea']

In [ ]:
ads[ads.name=='seizures>healthy']
# ads.loc[0, 'id']= 2274584

### 추가 전처리 필요없는 부작용들은  refined로 입력

In [ ]:
# remove_list.extend([')', '('])
# spe_chars = dict((re.escape(str(k)), ' ') for k in remove_list)
# sep_pattern = re.compile("|".join(spe_chars.keys()))
# int_pattern = re.compile('[0-9]')

# for ind, row in ads.iterrows():
#     refined = sep_pattern.sub(lambda m: spe_chars[re.escape(m.group(0))], row['name'])
#     int_refined = int_pattern.search(row['name'])
    
#     if refined.strip() == '':
#         ads.drop(ind, inplace=True)
#     elif row['name'].strip() == refined.strip() and int_refined is None:
#         ads.loc[ind]['refined'] = refined
#         print(row['name'])
#         print(ads.loc[ind]['refined'])
#     print('===============================')

In [ ]:
# ads.loc[ads.refined.str.len()>0].shape

In [ ]:
# ads.reset_index(drop=True)

In [ ]:
ads[ads.name.str.find('sudden death')>=0]

## 정체된 목록입니다.

In [ ]:
# ads.loc[ads.refined.str.strip()!=''].to_csv('./refined_ads_0226.csv', sep=',')

## 선생님께 확인 요청을 드리고 싶은 부분

In [ ]:
# ads[ads['refined'] == ''].to_csv('./need_to_refined_ads_0226.csv', sep=',')